# Week 11: Data Preprocessing

- 本周我们初步完成了将 68363 条从 StackOverflow 上爬取的数据进行预处理的工作
- 由于爬取数据时没有直接转为易于模型训练的 json 格式，而是使用 txt 用自定义的格式存储，所以为了方便后续的数据处理，我们需要将这些数据转为 json 格式（目标：能使用 `json.load(file.read())` 读取数据）
- 因为爬取数据时将每条帖子创建一个 txt 文件存储，所以导致 QA 文件夹内文件过多，不易管理，所以用 `merge_file` 将多个帖子存在一个文件中（之所以不一次性将所有帖子存在同一个文件，是为了方便 debug 检查 json 格式），并进行初步的格式化（包括调整引号和保留一个 solution 等）。
- 然后通过 `preprocessing_file` 尝试将多个文件的 data 合并到一个 List 中，并筛选掉空回复的帖子，然后转存为 json 文件。
- 最后就能将所有的数据存到一个 json 文件中，并确保能正确读取。结果：数据可以用 `json.load(file.read())` 读取，且可以转化为 DataFrame 等格式，方便模型使用。

Origin total data: 68363

Final total data: 51182

<font color=red>注意！</font> 运行本脚本必须确保将本文件置于与 qa 文件夹同目录的另一文件夹中！（如下）

```
|_qa
|
|_XXX
  |_preprocessing_data.ipynb
```

In [1]:
import json
import os
import re

In [ ]:
'''
merge_file.py
将每 1000 条帖子合并成一个文件，并将每个文件的数据格式化，对多个 solutions 的帖子，只筛选出第一条 solution
'''

'''
merge several files to one file, by the way formatting them to "json friendly" files
'''
def merge_txt_files(source_folder, output_folder, files_per_merge=1000):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    file_count = 0
    merge_count = 0
    outfile = None

    for root, _, files in os.walk(source_folder):  # 遍历 qa 的所有文件
        for file in files:
            if file.endswith('.txt'):
                if file_count % files_per_merge == 0:  # 当计数为 0 或满 1000 时，创建新文件
                    if outfile:
                        outfile.write("]\n")
                        print(f"File {output_file} created.")
                        outfile.close()
                    merge_count += 1
                    output_file = os.path.join(output_folder, f'merged_{merge_count}.json')
                    outfile = open(output_file, 'w', encoding='utf-8')  # 打开输出文件的输出流
                    outfile.write("[\n")
                else:
                    if outfile:
                        outfile.write(",\n")


                file_path = os.path.join(root, file)  # 输入文件的路径
                with open(file_path, 'r', encoding='utf-8') as infile:  # 打开输入文件的输入流
                    problem, solution = formatting(infile)  # 格式化文件内容
                    data = {"problem": problem, "solution": solution}  # 采用字典作为 json 输入元素
                    json.dump(data, outfile, ensure_ascii=False, indent=4)  # 将字典转为 json 格式写入输出文件

                file_count += 1

    if outfile:
        outfile.write("]\n")
        print(f"File {output_file} created.")
        outfile.close()

'''
Formatting the file content to json friendly format
'''
def formatting(file):
    mode = 0
    space_num = 0
    pbs = ""  # 存储问题
    slt = ""  # 存储回答
    while True:
        line = file.readline()
        if not line:
            break

        line = line.strip()
        # 将单引号转为双引号（json 的 key 必须是双引号）
        line = re.sub(r"'problems'", "\"problems\"", line)
        line = re.sub(r"'solutions'", "\"solutions\"", line)

        for match in re.finditer(r"('(.*)')|(\"(.*)\")", line):  # 检测最外围的单引号或双引号作为本行内容
            raw = match.group(0).strip()
            # 去除外围引号
            if raw[0] == "'" and raw[-1] == "'":
                raw = raw[1:-1]
            elif raw[0] == '"' and raw[-1] == '"':
                raw = raw[1:-1]
            
            # 去除内容中的 "\n"，替换为真正的换行符；对于多行换行符，转成一行
            if raw == "\\n":
                space_num += 1
                continue
            else:
                if space_num > 1:
                    raw = "\n" + raw
                space_num = 0

            if raw[-2:] == "\\n":
                raw = raw[:-2] + "\n"

            if raw == "solutions" or raw == "problems":
                if raw == "solutions":
                    mode = 1
                continue
            
            # 特殊处理（如果没有遇到内容中单引号变为“\'”的情况可以注释）
            raw = raw.replace("\\'", "'")
            
            # 对于多个 solutions，只取第一个
            if mode == 1 and raw == "............................................................":
                return pbs, slt

            if mode == 0:
                pbs += raw
            else:
                slt += raw

    return pbs, slt

# main
source_folder = '../qa'  # 保证文件位置
output_folder = 'merged_files'
merge_txt_files(source_folder, output_folder)

In [ ]:
'''
preprocessing_file.py
将多个文件的数据合并到一个 List 中，并筛选掉空回复的帖子，然后转存为 json 文件
'''
def filter_null_solution(src_folder, dst_folder, min_merge_per_file=1600):
    total_len = 0
    test_len = 0
    merge_num = 1
    if not os.path.exists(dst_folder):
        os.mkdir(dst_folder)
    outfile = open(f"{dst_folder}/fil_{merge_num}.json", "w", encoding="utf-8")
    tot_data = []

    for root, _, files in os.walk(src_folder):  # 遍历 merged_files 的所有文件
        for file in files:
            if test_len > min_merge_per_file:  # 确保每个文件至少有 min_merge_per_file 条数据
                print(f"File fil_{merge_num}.json has {test_len} items.")
                merge_num += 1
                test_len = 0
                json.dump(tot_data, outfile, ensure_ascii=False, indent=4)
                outfile.close()
                outfile = open(f"{dst_folder}/fil_{merge_num}.json", "w", encoding="utf-8")
                tot_data = []

            with open(os.path.join(root, file), "r") as f:
                data = json.loads(f.read())
            filtered_data = [item for item in data if item.get("solution")]  # 筛选掉空回复的帖子
            # print(f"{file}: {len(filtered_data)} data after filter.")
            test_len += len(filtered_data)
            total_len += len(data)
            tot_data.extend(filtered_data)

    print(f"File fil_{merge_num}.json has {test_len} items.")
    json.dump(tot_data, outfile, ensure_ascii=False, indent=4)
    outfile.close()
    print(f"Total data: {total_len}")

# main
filter_null_solution("merged_files", "filtered_files")

In [ ]:
'''
load_data.py
将所有数据存到一个 json 文件中，并确保能正确读取
'''
data = []
for root, _, files in os.walk("filtered_files"):
    for file in files:
        if file.endswith(".json"):
            with open(os.path.join(root, file), "r") as f:
                # print(f"Reading file {file}")
                data.extend(json.loads(f.read()))

print(f"Total data: {len(data)}")

# 将所有的数据存到一个 json 文件中
with open("all_data.json", 'w') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)